<a href="https://colab.research.google.com/github/Poulinakis-Konstantinos/Leaf-Disease-ML-detector-/blob/main/Plant_Disease_MobileNetV2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  Code for creating & Training a N.N. model with transfer learning based on MobileNetV2 . 

  The purpose of of the model is to classify plant's leaf images and extract decisions regarding their health. 

  It is able to classify 38 different classes ,13 different plants and some diseases that harm them . 

  The Model is on it's own very light , but we are still going to convert it into a TF Lite version  in order to run inference on a portable device , a RaspBerry Pi 4B . 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import files

In [ ]:
####### SCRIPT TO DOWNLOAD KAGGLE DATASET IN GOOGLE COLAB ######### .
from google.colab import files
#Upload your kaggle.json file 
files.upload() 

!mkdir -p  ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#Copy the API command from kaggle .
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

#Unzip dataset ,always with -q (quiet)
!unzip -q /content/new-plant-diseases-dataset.zip

In [ ]:
#Copy the API command from kaggle .
!kaggle datasets download -d vbookshelf/v2-plant-seedlings-dataset

#Unzip dataset ,always with -q (quiet)
!unzip -q /content/v2-plant-seedlings-dataset.zip

100% 3.19G/3.19G [00:57<00:00, 67.0MB/s]
100% 3.19G/3.19G [00:57<00:00, 59.8MB/s]


In [ ]:
image_size =224
batch_size =32
base_dir="/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"

In [ ]:
class_names=["Apple___Apple_scab","Apple___Black_rot","Apple___Cedar_apple_rust","Apple___healthy",
             "Blueberry___healthy","Cherry_(including_sour)__Powedery_mildew","Cherry_(including_sour)__healthy",
             "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot","Corn_(maize)___Common_rust_","Corn_(maize)___Northern_Leaf_Blight",
             "Corn_(maize)___healthy","Grape___Black_rot","Grape___Esca_(Black_Measles)","Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
             "Grape___healthy","Orange___Haunglongbing_(Citrus_greening)","Peach___Bacterial_spot","Peach___healthy",
             "Pepper,_bell___Bacterial_spot","Pepper,_bell___healthy","Potato___Early_blight","Potato___Late_blight",
             "Potato___healthy","Raspberry___healthy","Soybean___healthy","Squash___Powdery_mildew",
             "Strawberry___Leaf_scorch","Strawberry___Healthy","Tomato___Bacterial_spot","Tomato___Early_blight","Tomato___Late_blight",
             "Tomato___Leaf_Mold","Tomato___Septoria_leaf_spot","Tomato___Spider_mites Two-spotted_spider_mite",
             "Tomato___Target_Spot","Tomato_Yellow_Leaf_Curl_Virus","Tomato_mosaic_virus","Tomato___healthy"]

In [ ]:
# Train data set preparation, We apply data augmentation on-the-fly
#to increase generalizability of the model.
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255,
                                                            shear_range = 0.2,
                                                            zoom_range = 0.2,
                                                            width_shift_range = 0.2,
                                                            height_shift_range = 0.2,
                                                            fill_mode="nearest")

train_data = train_datagen.flow_from_directory(os.path.join(base_dir,"train"),
                                               target_size=(image_size,image_size),
                                               batch_size=batch_size,
                                               class_mode="categorical" 
                                               #save_to_dir='/content/aug_pic'                                        
                                              )


Found 70295 images belonging to 38 classes.


In [ ]:
# Validation data set preparation
valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)
valid_data = valid_datagen.flow_from_directory(os.path.join(base_dir,"valid"),
                                               target_size=(image_size,image_size),
                                               batch_size=batch_size,
                                               class_mode="categorical"                                              
                                              )

Found 17572 images belonging to 38 classes.


In [ ]:
########### CREATING THE NN MODEL (TRANSFER LEARNING) ############
from tensorflow.keras.applications import MobileNetV2 #14 MB acc Top1 0.71 top5 0.9
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model . Load pretrained weights + chop off head of the network(classifier) . 
base_model_MobileNetV2 = MobileNetV2(weights='imagenet',
                               include_top=False ,input_shape=(224,224,3))

def create_model(base_model):
      # add a global spatial average pooling layer
      x = base_model.output
      x = GlobalAveragePooling2D()(x)
      # let's add a fully-connected layer
      x = Dense(1024, activation='relu')(x)
      # and a logistic layer -- let's say we have #number classes
      predictions = Dense(38, activation='softmax')(x)

      # this is the model we will train
      model = Model(inputs=base_model.input, outputs=predictions 
                                      ,name="Plant_Disease_Detector")

      # first: train only the top layers (which were randomly initialized)
      # i.e. freeze all convolutional MobileNet layers
      for layer in base_model.layers:
          layer.trainable = False

      base_learning_rate = 0.0001
      # compile the model (should be done *after* setting layers to non-trainable)
      model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate)
                    , loss='categorical_crossentropy' ,metrics='accuracy')
      
      return model
      
model_M = create_model(base_model_MobileNetV2)

In [ ]:
model_M.summary()

In [ ]:
########   TRAIN   ##########
  ##MOBILEV2 net##
history=model_M.fit(x=train_data ,batch_size=batch_size,
          epochs=30 ,verbose=1 ,validation_data=valid_data,
           steps_per_epoch=150 ,validation_steps=100)

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model_MobileNetV2.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model_M.layers[:249]:
   layer.trainable = False
for layer in model_M.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model_M.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
                metrics='accuracy',loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history_M2=model_M.fit(x=train_data ,batch_size=batch_size,
          epochs=10 ,verbose=1 ,validation_data=valid_data,
           steps_per_epoch=150 ,validation_steps=100)

In [ ]:
###### CONVERT THE MODEL #######
###  Converting the model to it's tflite equivalent will allow
###  it to run efficiently on a Rasberry Pi for live detection . 
converter = tf.lite.TFLiteConverter.from_keras_model(model_M)

#converter.optimizations = [tf.lite.Optimize.DEFAULT] ##Quantized
tflite_model = converter.convert()

# Save the model.
with open('model.tfliteQuant', 'wb') as f:
  f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpmn12orh7/assets


In [ ]:
####### MANUAL WEIGHT SAVE #######                    ### For my expirementations
model_M.save_weights("./MobileNetV2_Weights.h5")
files.download("./MobileNetV2_Weights.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
####### RESTORING MODEL'S WEIGHTS ###########          ### For my expirementations
model_M.load_weights('./MobileNetV2_Weights.h5')
# Evaluate the model
##model_clone.evaluate(valid_data)

In [ ]:
############  SAVING THE WHOLE MODEL  ###########        ### For my expirementations
!mkdir -p saved_model 
model_M.save('saved_model/MobileNetV2') 

# my_model directory
!ls saved_model
# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/MobileNetV2
# Zip it before download (So that you can download the whole directory at once )
!zip  -r /content/MobileNetV2_In_Zip.zip /content/saved_model
print("Zip Ready")
# Download it 
files.download('/content/MobileNetV2_In_Zip.zip')

In [ ]:
############  RESTORING THE WHOLE MODEL ###########         ### For my expirementations
files.upload()

!unzip -q /content/saved_model_In_Zip.zip 
new_model = tf.keras.models.load_model('/content/content/saved_model/MobileNetV2')
# Check its architecture
new_model.summary()

In [ ]:
######  PLOTTING TRAINING INFO ########
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

fig = plt.figure(figsize=(10,6))
plt.plot(epochs,loss,c="red",label="Training Loss")
plt.plot(epochs,val_loss,c="blue",label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

fig = plt.figure(figsize=(10,6))
plt.plot(epochs,acc,c="red",label="Training Acc")
plt.plot(epochs,val_acc,c="blue",label="Validation Acc")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

In [ ]:
####### SCRIPT TO MAKE PREDICTIONS ON THE TEST SET DIRECTORY ######

## I have written this script in order to automate my model evaluation on a test dataset .  

import pathlib
path='/content/test/test'
i=0
#Inversing class dictionary [keys,values]->[values,keys]
labels = (train_data.class_indices)
labels = dict((v,k) for k,v in labels.items())

for filename in os.listdir(path) :
    i+=1
    file_path= path +"/" + str(pathlib.Path(filename))
   
    image = tf.keras.preprocessing.image.load_img(
            file_path,grayscale=False, color_mode="rgb",
            target_size=(image_size,image_size), interpolation="nearest")
    input_arr = keras.preprocessing.image.img_to_array(image)
    input_arr=input_arr/255
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    prediction = model_M.predict(input_arr)


    a=np.argmax(prediction)
    results = labels[a] 

    
    print("\n")
   # print("Predicted class number :", np.argmax(prediction))
    print("Actual class :" ,pathlib.Path(filename))
    print("predicted class name->  MobileV2:",results )
    

print("\n Tested: " ,i, "new unseen images")

In [ ]:
eval=new_model.evaluate(valid_data)
print(eval)

550/550 [==============================] - 60s 96ms/step - loss: 0.2129 - accuracy: 0.9318
[0.21291173994541168, 0.9317664504051208]
